In [1]:
#import gensim
from gensim import corpora, models, similarities, utils
from pprint import pprint   # pretty-printer
import logging
import numpy as np
import pandas as pd
import nltk
import pymysql
import re
import pyLDAvis.gensim
import pyLDAvis

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
def call_mysql():
    # pull in data from the reviews table
    con = False
    rows = np.nan
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            sql = 'USE yelpdata;'
            cur.execute(sql)
            sql = 'SELECT id,business_id,stars,comment FROM review;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
#             for row in rows:
#                 print(row)
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [4]:
rows = call_mysql()

# UNDERSTAND THE OUTPUT OF THE MYSQL
# print(np.shape(rows))#(3616, 4)
# print(type(rows))#tuple
# print(type(rows[:][0]))# still tuple
# print(type(rows[3][3]))# int/str or whatever per the column

#print(rows[0][3])#just the review for the zero-indexed row [0]
# print(rows[1][3])#the review for the one-indexed row [1]

# confusing - not printing all rows of the first column
#print(rows[:][0])#all four columns of the data-row

documents = []#type list
ndocs = np.shape(rows)[0]

#reviews = [[review for review in rows[3]] for row in rows]
for row in rows:
    documents.append(row[3])# row[3] is type string, documents is type list

print(documents[0])# prints one review

# for doc in range(0,ndocs):
#     [review for review in rows[:][0]]# This prints the review from the id in the second []
#     print review#type str

# print(row[3])#just the review
# print(row[:])# a row of data
# print(review)# one review


Adding some more information. Another great thing about him is that he was willing to experiment with me for my fibromyalgia. I brought in a book by a specialist and he was willing to try the protocol. Many doctors would not do that. He won't do anything crazy but will consider something reasonable. He also made himself available when I was having an emergency severe toothearache. He was at an offsite seminar, but his receptionist asked him to call me and he did, prescribed something and I think he even checked in with me later on or I called back. Either way, I got the treatment I needed.For those who think it takes a while to get ahold of him or his receptionistplease keep in mind it is one doctor and one receptionist. As you can see from all of the 5 star reviews, he spends time with his patients. Be a little patient yourself and you will be soon benefit from his great treatment.


In [5]:
def unify_time(word):
    #Assign all possible time values ('9:45','12:05',etc) to one word 'oclock'
    match = re.search(r'\d*'+':'+r'\d*',word)
    if match:
        #print(match.group())
        word = 'oclock'
    return word


    # This also works, better if you want to catch the error and do something, but I don't
    # try: 
    #     match = re.search(r'\d*'+':'+r'\d*',s, word)
    #     test = match.group() 
    #     word = 'oclock'
    # except AttributeError as e: 
    #     print('error in unify_time')

In [6]:
def period_split(word):
    # Split words with a period in the middle - not yet the ideal implementation b/c it returns two words as one
    match = re.search(r'\w*'+'.'+r'\w*',word)
    if match:
        word = word.replace('.',' ')
    return word

In [7]:
def canonicalize(word):    
    bads = ['dr.',"n't","'ve","'re",'wo',"'m"]
    goods = ['dr','not','have','are','will','am']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

In [8]:
def unify_pronouns(word):
    """ map many onto one for pronouns - parts of speech"""
    many = ['he',"his","him","she",'her',"hers",'I','i','me','mine','my','you','your',
            'yours','it','its','they','their','theirs','we','us','our','ours']
    few = ['male','male','male','female','female','female','firstper','firstper',
           'firstper','firstper','firstper','secper','secper','secper',
           'neut3rd','neut3rd','neut3rd','neut3rd','neut3rd',
          'coll3rd','coll3rd','coll3rd','coll3rd']
    try:
        idx = many.index(word)
        word=few[idx]
    except ValueError as e:
        word=word
    return word

In [9]:
def clean(word):
    # wrapper function - performs a bunch of functions on a given word
    word = unify_time(word)
    word = unify_pronouns(word)
    word = canonicalize(word)
    word = period_split(word)    
    return word

In [10]:
def numwords():
    #add numbers to common words
    nums = np.arange(0,100)
    ss =''
    ss = ss.join([' '+str(n) for n in nums])
    return ss

In [170]:
def query_SQL(sql):
    con = False
    rows = []
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            database = 'USE yelpdata;'
            cur.execute(database)
            #sql = 'SELECT DISTINCT(yelp_id) FROM business;' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
            #print('business id from table: ',rows, type(rows))
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [171]:
def ids_to_names():
# Purpose of function is to exclude proper names of doctors - first and last names
# Function will accept list of yelp_ids extracted by mysql from business table
# Function will put together a list of names
# Function will exclude terms like family doctor and pediatrics

    dashed_names=[]#examples = ['tamalpais-pediatrics-novato','julie-doctor-kim']
    sql = 'SELECT DISTINCT(yelp_id) FROM business;' 
    yelp_ids = query_SQL(sql)# extracts unique yelp_ids
    for i in range(0,len(yelp_ids)):
        dashed_names.append(yelp_ids[i][0])
    
    def split_append(yelp_id):
        yelp_id = yelp_id.split('-')
        #print(yelp_id)
        return yelp_id

#     business_type = ['pediatrics','family','medicine','doctor','institute','center',
#                      'practice','md','san','francisco','ucsf','one','medical','group',
#                     'sports','fitness','therapy','acupuncture','chiropractic','district']
    
    names = []
#     for name in dashed_names:
#         [names.append(l) for l in split_append(name) if l not in business_type]
    for name in dashed_names:
        [names.append(l) for l in split_append(name)]
    return names

In [123]:
def get_pat_for_tokenizer():
    from nltk.tokenize import regexp_tokenize
    """money and time patterns formatted to fit in nltk RegexpTokenizer object, debugged with s2 below"""
    #s2 = 'it took until 12:30 am to get an appointment and I spent $3.50'
    
    pat = '\w+|\$[\d\.]+|\S+'
    pat2 = '\d+\:[\d]+\s?(pm|am)|\S+'

    pat3 = '('+pat2+')' +'|'+ '('+pat+')'#must be one string argument... | is OR
    #tokenizer = RegexpTokenizer(pat3)#time
    return pat3

In [129]:
# Two ways to clean & tokenize - stoplists, lowercase, regular expressions, time-expressions

# from nltk.corpus import stopwords
# from nltk.tokenize import RegexpTokenizer

#texts: A list of lists; text: list of tokenized words from 1 doc/review
#document = a string

# 1) uses my stoplist - see above
# texts = [[clean(word) for word in nltk.word_tokenize(document.lower()) if word not in stoplist]
#         for document in documents]#review in reviews 

#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')#money
#tokenizer = RegexpTokenizer('\d*'+'\:'+'\d*')#time
tokenizer = RegexpTokenizer(get_pat_for_tokenizer())#time & money
mystopwords = stopwords.words() + ids_to_names() + ['dr.','dr','doctor','doctors',"doctor's","doctors'",'get']#type list
#print(mystopwords[-500:])

In [130]:
# 2) uses nltk functions
# texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower()) if word not in stopwords.words()]
#         for document in documents]#review in reviews 

texts = [[canonicalize(word) for word in tokenizer.tokenize(document.lower()) if word not in mystopwords]
        for document in documents]#review in reviews 

#print(texts[:2])

In [131]:
# BAG OF WORDS
dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts
#dictionary.save('/Users/kaschbacher/insightproject/yelp/git-yelp/doctor.dict') # store the dictionary, for future reference
print(dictionary)
#print(type(dictionary))#<class 'gensim.corpora.dictionary.Dictionary'>
#print(dictionary.token2id)# very long!

# To merge with another dictionary -  Dictionary.merge_with())# Apply bag of words to reviews & print tokenized dictionary

Dictionary(31602 unique tokens: [u'considered,', u'hanging', u'ovaries,', u'aileen', u'ovaries.']...)


In [132]:
# SERIALIZE & SAVE TO DISK 

# Convert tokenized documents to sparse vectors: -- [(0, 1), (4, 3),...
# new_vec = dictionary.doc2bow(documents[1].lower().split())
# print(new_vec)

corpus = [dictionary.doc2bow(text) for text in texts]
corpus_mm = corpora.MmCorpus.serialize('doctor_bow_for_lda.mm', corpus) # store to disk, for later use

#print(corpus)
# looks like - [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)... (35, 2)...
# (word order index, frequency of occurence?)

In [133]:
# LDA  (takes awhile to run)
# The constructor estimates Latent Dirichlet Allocation model parameters based on a training corpus 
# id2word is a mapping from word ids (integers) to words (strings). 

# 1) The constructor - obtains LDA model parameters given training corpus
lda = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10, update_every=0, passes=10, iterations=100)


In [136]:
# SAVE THE LDA OBJECT
# --- note - i tried to do this with gensim, but it wasn't working, and ultimately cpickle did
#from gensim.utils import SaveLoad
#save(fname_or_handle, separately=None, sep_limit=10485760, ignore=frozenset([]), pickle_protocol=2)
#utils.SaveLoad.save(lda, pickle_protocol=2)

import cPickle as pickle
pickle_out = open('lda.p', 'wb')
pickle.dump(lda, pickle_out)
pickle_out.close()

In [137]:
# PRINT THE TOPICS
lda.top_topics(corpus,num_words=10)

[([(0.0098564377634083253, u'surgery'),
   (0.00644469875339707, u'time'),
   (0.0055493912592786531, u'see'),
   (0.0047668046741630027, u'first'),
   (0.0047634426718445562, u'would'),
   (0.0047628181654309669, u'really'),
   (0.0046104815944286934, u'great'),
   (0.0040553088997897493, u'like'),
   (0.0037792626647346578, u'could'),
   (0.0036644022761603444, u'office')],
  -88.921650813424193),
 ([(0.0053799532231160923, u"i've"),
   (0.0047461826395626839, u'office'),
   (0.004664561047390426, u'would'),
   (0.0045972694299512137, u'never'),
   (0.0045927850071772573, u'see'),
   (0.0041284197730383073, u'time'),
   (0.0040382454918490825, u'surgery'),
   (0.0039099164961633529, u'went'),
   (0.0038281434053625621, u'really'),
   (0.0037924116552917938, u'first')],
  -95.103441120641136),
 ([(0.0081734184887351305, u'time'),
   (0.0065141351110002479, u'office'),
   (0.0064867422583847296, u'would'),
   (0.0061540773116233486, u'like'),
   (0.0053169797873792455, u'see'),
   (0.0

In [147]:
# GAMMAS = matrix: REVIEWS BY TOPICS
gammas,_ = lda.inference(corpus)# first element returned by lda.inference is gammas matrix: #shape (3616,10)
# Normalize:  sum over topics should equal one
norm_gammas = gammas/(np.sum(gammas,axis=1)[:,None])# Axis=1 = sum over columns for each row; Axis=0 sum over rows in each column
print(norm_gammas[0][:])
#print(sum(norm_gammas[0][:]))#Yes, check that =1
#print(np.shape(norm_gammas))#(3616,10)
norm_gammas_df = pd.DataFrame(norm_gammas)

# SAVE THE NORM_GAMMAS DATAFRAME OBJECT

import cPickle as pickle
pickle_out = open('norm_gammas_df.p', 'wb')
pickle.dump(norm_gammas, pickle_out)
pickle_out.close()

[ 0.00149286  0.9865636   0.00149295  0.00149301  0.00149295  0.00149291
  0.00149299  0.0014928   0.00149293  0.00149301]


In [175]:
# What reviews best represent a given topic?
# Those with the highest gamma-weights on one topic (and the lowest cross-vocality on other topics)

#print('Original matrix:',norm_gammas_df.loc[:5,:5])
# Sort on a given topic
col = 1
mysorted = norm_gammas_df.sort(col, axis=0, ascending=False)
print('Top 5 documents related to topic ',(col+1),': ')
#print(mysorted.loc[:2,:5])
idx = mysorted.index[0]
print('Index of document with the top score on a given topic (from col): ',idx)

# index in the dataframe is not equivalent to id in the review table. 669 is the first ID
review_id = str(669+idx)
print(review_id)
sql = 'SELECT comment FROM review WHERE id='+review_id+';'
rows = query_SQL(sql)

print(rows)

('Top 5 documents related to topic ', 2, ': ')
('Index of document with the top score on a given topic (from col): ', 1298)
1967
(("Four years ago I was sitting in the room of my aesthetic nurse specialist in Walnut Creek California when I noticed how beautiful she appeared. For the previous five years she had done an amazing job at helping me to age gracefully with various skin treatments and lasers. Something looked very different about my friend , she looked ten years younger!  After relentless prying on my part she informed me of her recent face lift. Knowing that she was married to a prominent doctor herself I was eager to find out the identity of the surgeon. To my surprise it was not her employer but a doctor in San Francisco. She quietly pulled out the golden egg the card of Dr. Issa Eshima. I soon discovered that Dr. Eshima is the surgeon that fellow doctors call upon when their wives are in need of aesthetic refreshment. While most surgeons will do everything Dr. Eshima is un

In [139]:
vis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

In [32]:
pickle.dump(vis_data, open('vis_data.p','wb'))